In [ ]:
import os
import json
import pandas as pd
from collections import defaultdict

In [ ]:
!unzip "/content/Automatic Hallucination Detection Results.zip" -d /content/hallucinations

In [12]:
import os
import json
import pandas as pd
from collections import defaultdict

FOLDER = "/content/hallucinations/Automatic Hallucination Detection Results"

# Load json files into a list of lists
def load_json_files(folder):
    all_results = []
    for fname in os.listdir(folder):
        if fname.lower().endswith(".json"):
            path = os.path.join(folder, fname)
            with open(path, "r", encoding="utf-8") as f:
                try:
                    data = json.load(f)
                    all_results.append(data)
                except Exception as e:
                    print(f"Failed to load {fname}: {e}")
    return all_results

# Compute total hallucinations and total counts by hallucination type
def compute_metrics(all_results):
    total_count_by_type = defaultdict(int)
    total_hallucinations = 0

    for result_list in all_results:
        for item in result_list:
            etype = item.get("Error Type", "Unknown")
            c = item.get("Count", 0)
            total_count_by_type[etype] += c
            total_hallucinations += c

    return total_hallucinations, total_count_by_type

# Run processing
all_results = load_json_files(FOLDER)
total_hallucinations, total_count_by_type = compute_metrics(all_results)

# Convert type counts to DataFrame
type_df = pd.DataFrame(
    [{"Hallucination Type": etype, "Total Count": count}
      for etype, count in total_count_by_type.items()]
).sort_values("Total Count", ascending=False)


# Output results
print(f"Total Hallucinations Across All Files: {total_hallucinations}\n")
print("Hallucination Distribution by Type:")
print(type_df)

# Save CSV
type_df.to_csv("/content/hallucination_type_distribution.csv", index=False)
print("\nSaved: /content/hallucination_type_distribution.csv")


Total Hallucinations Across All Files: 29

Hallucination Distribution by Type:
       Hallucination Type  Total Count
0        Unsupported Fact           20
1  Unsupported Medication            6
3        Unsupported Time            2
2   Unsupported Procedure            1

Saved: /content/hallucination_type_distribution.csv
